In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Task 1

For this task i am required to scrape customer reviews on british airways from <a href='https://www.airlinequality.com'>Skytrax</a>,clean it and analyse the data (sentiment analysis and word clouds were suggested)

## Scraping data from Skytrax
We can find customer reviews on the airline<a href='https://www.airlinequality.com/airline-reviews/british-airways'>here</a>.
I use beautiful soup to extract it

Importing libraries.............

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

This script extracts 1500 reviews from the website

In [3]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 15 
page_size = 100 #How many reviews we intend to scrape per page
reviews = [] #list to hold all our reviews


max_retries = 7 #How many times we wan't our script to try scrape the data [ poor network problems :( ]

for i in range(1, pages + 1):
    print(f"Currently scraping page {i}") #Let's me know my script actually works and how far it's gone

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    retries = 0  # Initialize the retry counter

    while retries < max_retries: #retries till it has made 7 attempts
        try:
            
            response = requests.get(url)

            # Check if the response status code is 200 (OK)
            if response.status_code == 200:
        
                content = response.content
                soup = BeautifulSoup(content, 'html.parser') #Creating the soup that contains all our text

                for para in soup.find_all("div", {"class": "text_content"}):
                    reviews.append(para.get_text()) #appends the reviews we find into a reviews list

                print(f"   ---> {len(reviews)} total reviews") #tells me how many reviews i have so far
                break  # Exit the retry loop if successful

        except requests.exceptions.RequestException as e:
            print(f"Error retrieving page {i} - Retry {retries + 1}: {e}")

        # Increment the retry counter
        retries += 1 #increases the number of tries

    if retries == max_retries: #to let me know my network is terrible or the website has issues
        print(f"Failed to retrieve page {i} after {max_retries} retries.") 


Currently scraping page 1
   ---> 100 total reviews
Currently scraping page 2
   ---> 200 total reviews
Currently scraping page 3
   ---> 300 total reviews
Currently scraping page 4
   ---> 400 total reviews
Currently scraping page 5
   ---> 500 total reviews
Currently scraping page 6
   ---> 600 total reviews
Currently scraping page 7
   ---> 700 total reviews
Currently scraping page 8
   ---> 800 total reviews
Currently scraping page 9
   ---> 900 total reviews
Currently scraping page 10
   ---> 1000 total reviews
Currently scraping page 11
   ---> 1100 total reviews
Currently scraping page 12
   ---> 1200 total reviews
Currently scraping page 13
   ---> 1300 total reviews
Currently scraping page 14
   ---> 1400 total reviews
Currently scraping page 15
   ---> 1500 total reviews


Now to load the data into a pandas dataframe.............

In [4]:
df = pd.DataFrame({'Reviews': reviews})
df.head()

,Reviews
0,✅ Trip Verified | First time flying British Ai...
1,Not Verified | I flew London to Cairo and ret...
2,Not Verified | Absolutely the worst experienc...
3,Not Verified | Flew back from Malta after sc...
4,Not Verified | Cabin luggage had to go to carg...


We notice the reviews have verified or not verified before hand, this is uneccesary, so to remove it...........

In [5]:
df['Reviews'] = df['Reviews'].str.split('|',expand=True)[1] 
#Splits the text in each row and selects the second split only
df.head()

,Reviews
0,First time flying British Airways and I would...
1,I flew London to Cairo and return in October...
2,Absolutely the worst experience ever. Flew ...
3,Flew back from Malta after scattering our s...
4,"Cabin luggage had to go to cargo, even when I..."


looking good